<a href="https://colab.research.google.com/github/alexei-guirik/PythonUrbanITMO2023/blob/main/%D0%93%D0%B8%D1%80%D0%B8%D0%BA_%D0%90%D0%92_PythonUrban_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [105]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify osmnx

In [106]:
# TODO собрать импорты всех модулей в одном месте
import osmnx as ox

from google.colab import output
output.enable_custom_widget_manager()

In [107]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Приморский район, Санкт-Петербург'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = 'https://raw.githubusercontent.com/alexei-guirik/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson'  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL =  'https://raw.githubusercontent.com/alexei-guirik/PythonUrbanITMO2023/main/geojson_layers/streets.geojson'  # ссылка на слой с улицами
#KGIOP_FILE = 'kgiop_objects.geojson'  # Файл с объектами культурного наследия
#STREETS_FILE =  'streets.geojson'  # Файл с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [108]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf(TERRITORY_NAME)
#district

## Улицы

### Загрузка файла с улицами (Extract)

In [109]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
routes_graph = ox.graph_from_place(TERRITORY_NAME, network_type='all')
routes_graph
roads = ox.graph_to_gdfs(routes_graph, nodes=False)
roads.reset_index(drop=True, inplace=True)
roads

,osmid,oneway,lanes,name,highway,maxspeed,reversed,length,bridge,geometry,width,ref,junction,access,service,tunnel
0,"[68711174, 166554665, 145439671, 209040120, 14...",True,3,"[1-й Лахтинский мост, Приморское шоссе]",primary,"[40, RU:urban]",False,951.833,yes,"LINESTRING (30.18597 59.99033, 30.18582 59.990...",NaN,NaN,NaN,NaN,NaN,NaN
1,309915526,True,3,Приморский проспект,secondary,RU:urban,False,80.294,NaN,"LINESTRING (30.23119 59.98480, 30.23167 59.984...",NaN,NaN,NaN,NaN,NaN,NaN
2,10503391,True,2,Липовая аллея,secondary,RU:urban,False,9.232,NaN,"LINESTRING (30.25651 59.98305, 30.25651 59.98314)",NaN,NaN,NaN,NaN,NaN,NaN
3,775689580,True,3,Приморский проспект,primary,RU:urban,False,112.887,NaN,"LINESTRING (30.25651 59.98305, 30.25577 59.983...",NaN,NaN,NaN,NaN,NaN,NaN
4,22634234,False,2,улица Оскаленко,residential,RU:urban,False,7.804,NaN,"LINESTRING (30.27829 59.98310, 30.27827 59.98317)",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91977,"[1220309844, 1220309845]",False,NaN,NaN,"[steps, footway]",NaN,False,8.498,NaN,"LINESTRING (30.22540 60.00759, 30.22545 60.007...",NaN,NaN,NaN,NaN,NaN,NaN
91978,1168162509,False,NaN,NaN,service,NaN,True,13.559,NaN,"LINESTRING (30.23139 60.03566, 30.23152 60.03556)",NaN,NaN,NaN,NaN,NaN,NaN
91979,1221606756,False,NaN,NaN,footway,NaN,False,161.842,NaN,"LINESTRING (30.23139 60.03566, 30.23390 60.03640)",NaN,NaN,NaN,NaN,NaN,NaN
91980,1221606756,False,NaN,NaN,footway,NaN,True,175.986,NaN,"LINESTRING (30.23139 60.03566, 30.22867 60.03486)",NaN,NaN,NaN,NaN,NaN,NaN


### Обработка данных с улицами (Transform)

In [110]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
roads = roads[~roads['name'].apply(lambda x: isinstance(x, list))]
roads = roads.dissolve(by='name')
roads

,geometry,osmid,oneway,lanes,highway,maxspeed,reversed,length,bridge,width,ref,junction,access,service,tunnel
name,,,,,,,,,,,,,,,
1-й Озерковский переулок,"MULTILINESTRING ((30.29811 60.03748, 30.29658 ...",105053150,False,1,residential,None,False,94.655,None,None,None,None,None,None,None
1-й проезд,"MULTILINESTRING ((30.28057 60.03949, 30.28040 ...",132600765,False,None,residential,None,True,74.552,None,None,None,None,None,None,None
1-я Алексеевская улица,"MULTILINESTRING ((30.29409 60.01656, 30.29333 ...",36771653,False,1,residential,RU:urban,False,108.278,None,None,None,None,None,None,None
1-я Конная Лахта,"MULTILINESTRING ((30.12648 60.00618, 30.12585 ...",36945763,False,2,secondary,RU:urban,False,69.684,None,None,None,None,None,None,None
1-я Никитинская улица,"MULTILINESTRING ((30.28669 60.01860, 30.28687 ...",31399507,False,1,residential,RU:urban,False,10.141,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
улица Уточкина,"MULTILINESTRING ((30.26147 60.00788, 30.26154 ...",651213152,True,3,tertiary,RU:urban,False,57.166,None,None,None,None,None,None,None
улица Чёрная Речка,"MULTILINESTRING ((30.00172 60.01225, 30.00135 ...","[163783682, 163783683, 162991583]",False,None,residential,RU:urban,False,163.677,yes,None,None,None,None,None,None
улица Шаврова,"MULTILINESTRING ((30.23911 60.02527, 30.23892 ...",124801390,False,2,tertiary,RU:urban,True,11.974,None,None,None,None,None,None,None


### Сохранение слоя с улицами (Load)

In [114]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    #"name": "Название",
    "osmid": "Идентификатор",
    "oneway": "Односторонняя",
    "lanes": "Количество полос",
    "highway": "Тип дороги",
    "maxspeed": "Максимальная скорость",
    "reversed": "Нумерация в обратную сторону",
    "length": "Длина",
    "geometry": "geometry"
}
roads.rename(columns=rename_columns, inplace=True)
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
roads.index.rename('Название', inplace=True)
roads = roads[rename_columns.values()]              # Оставим только переименованные столбцы
roads


,Идентификатор,Односторонняя,Количество полос,Тип дороги,Максимальная скорость,Нумерация в обратную сторону,Длина,geometry
Название,,,,,,,,
1-й Озерковский переулок,105053150,False,1,residential,None,False,94.655,"MULTILINESTRING ((30.29811 60.03748, 30.29658 ..."
1-й проезд,132600765,False,None,residential,None,True,74.552,"MULTILINESTRING ((30.28057 60.03949, 30.28040 ..."
1-я Алексеевская улица,36771653,False,1,residential,RU:urban,False,108.278,"MULTILINESTRING ((30.29409 60.01656, 30.29333 ..."
1-я Конная Лахта,36945763,False,2,secondary,RU:urban,False,69.684,"MULTILINESTRING ((30.12648 60.00618, 30.12585 ..."
1-я Никитинская улица,31399507,False,1,residential,RU:urban,False,10.141,"MULTILINESTRING ((30.28669 60.01860, 30.28687 ..."
...,...,...,...,...,...,...,...,...
улица Уточкина,651213152,True,3,tertiary,RU:urban,False,57.166,"MULTILINESTRING ((30.26147 60.00788, 30.26154 ..."
улица Чёрная Речка,"[163783682, 163783683, 162991583]",False,None,residential,RU:urban,False,163.677,"MULTILINESTRING ((30.00172 60.01225, 30.00135 ..."
улица Шаврова,124801390,False,2,tertiary,RU:urban,True,11.974,"MULTILINESTRING ((30.23911 60.02527, 30.23892 ..."


In [62]:
# TODO сохранить слой в географической проекции в формате GeoJSON
#roads.to_crs(4326).to_file(STREETS_FILE, driver='GeoJSON')

ValueError: ignored

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [73]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
#tags = {'amenity': ['arts_centre', 'exhibition_centre', 'theatre']}         # Не совсем понял, что подразумевается под объектами культурного наследия, взял такие
tags = {'historic': True}                                                    # Не совсем понял, что подразумевается под объектами культурного наследия, взял такие
objects = ox.features.features_from_polygon(district.unary_union, tags)
objects

historic  \
element_type osmid                  
node         538426140   memorial   
             679667984   memorial   
             818774996   memorial   
             1066527082  memorial   
             1145235093  memorial   
...                           ...   
way          103263767      manor   
             113380025      ruins   
             163902553   monument   
             566202597        yes   
relation     1566440          yes   

                                                               inscription  \
element_type osmid                                                           
node         538426140   Военным морякам тихоокеанцам, погибшим при исп...   
             679667984   Площадь названа в честь авиаконструктора Игоря...   
             818774996                                                 NaN   
             1066527082                                                NaN   
             1145235093                                                NaN   
...                                                                    ...   
way          103263767                                                 NaN   
             113380025                                                 NaN   
             163902553                                                 NaN   
             566202597                                                 NaN   
relation     1566440                                                   NaN   

                        material  memorial  \
element_type osmid                           
node         538426140     stone     stone   
             679667984       NaN    plaque   
             818774996     stone  memorial   
             1066527082      NaN       NaN   
             1145235093      NaN     stone   
...                          ...       ...   
way          103263767       NaN       NaN   
             113380025       NaN       NaN   
             163902553       NaN       NaN   
             566202597       NaN       NaN   
relation     1566440         NaN       NaN   

                                                               name  \
element_type osmid                                                    
node         538426140                  Военным морякам-тихоокенцам   
             679667984                             И. И. Сикорскому   
             818774996           Закладной камень Аллеи журналистов   
             1066527082       Ликвидаторам Чернобыльской катастрофы   
             1145235093               Камень памяти Ольги Берггольц   
...                                                             ...   
way          103263767   Охотничий домик графа В.А. Стенбок-Фермора   
             113380025             Пороховой погреб с лабораториями   
             163902553                                          NaN   
             566202597                                          NaN   
relation     1566440                         Храм Дацан Гунзэчойнэй   

                                                                  geometry  \
element_type osmid                                                           
node         538426140                           POINT (30.26416 59.99768)   
             679667984                           POINT (30.26373 60.01322)   
             818774996                           POINT (30.20110 60.00931)   
             1066527082                          POINT (30.23274 60.01323)   
             1145235093                          POINT (30.29266 59.98902)   
...                                                                    ...   
way          103263767   POLYGON ((30.13090 59.99017, 30.13143 59.99012...   
             113380025   POLYGON ((29.96512 60.01324, 29.96526 60.01345...   
             163902553   POLYGON ((30.20075 59.98154, 30.20077 59.98154...   
             566202597   POLYGON ((30.14769 59.99422, 30.14780 59.99422...   
relation     1566440     POLYGON ((30.25566 59.98372, 30.25630 59.983

### Обработка объектов культурного наследия (Transform)

In [74]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
objects["lon"] = objects.to_crs(3857).geometry.centroid.to_crs(4326).x
objects["lat"] = objects.to_crs(3857).geometry.centroid.to_crs(4326).y

### Сохранение слоя с объектами культурного наследия (Load)

In [79]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "historic": "Тип",
    "inscription": "Надпись",
    "material": "Материал",
    "name": "Название",
    "lon": "Долгота",
    "lat": "Широта"
}
objects.rename(columns=rename_columns, inplace=True)
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
objects.index.rename(['Тип элемента', 'Идентификатор'], inplace=True)
objects = objects[['Тип', 'Надпись', 'Материал', 'Название', 'Долгота', 'Широта', 'geometry']]         # Оставим только переименованные столбцы
objects

<ipython-input-79-7c9ed28e6e6b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  objects.rename(columns=rename_columns, inplace=True)


Тип  \
Тип элемента Идентификатор             
node         538426140      memorial   
             679667984      memorial   
             818774996      memorial   
             1066527082     memorial   
             1145235093     memorial   
...                              ...   
way          103263767         manor   
             113380025         ruins   
             163902553      monument   
             566202597           yes   
relation     1566440             yes   

                                                                      Надпись  \
Тип элемента Идентификатор                                                      
node         538426140      Военным морякам тихоокеанцам, погибшим при исп...   
             679667984      Площадь названа в честь авиаконструктора Игоря...   
             818774996                                                    NaN   
             1066527082                                                   NaN   
             1145235093                                                   NaN   
...                                                                       ...   
way          103263767                                                    NaN   
             113380025                                                    NaN   
             163902553                                                    NaN   
             566202597                                                    NaN   
relation     1566440                                                      NaN   

                           Материал  \
Тип элемента Идентификатор            
node         538426140        stone   
             679667984          NaN   
             818774996        stone   
             1066527082         NaN   
             1145235093         NaN   
...                             ...   
way          103263767          NaN   
             113380025          NaN   
             163902553          NaN   
             566202597          NaN   
relation     1566440            NaN   

                                                              Название  \
Тип элемента Идентификатор                                               
node         538426140                     Военным морякам-тихоокенцам   
             679667984                                И. И. Сикорскому   
             818774996              Закладной камень Аллеи журналистов   
             1066527082          Ликвидаторам Чернобыльской катастрофы   
             1145235093                  Камень памяти Ольги Берггольц   
...                                                                ...   
way          103263767      Охотничий домик графа В.А. Стенбок-Фермора   
             113380025                Пороховой погреб с лабораториями   
             163902553                                             NaN   
             566202597                                             NaN   
relation     1566440                            Храм Дацан Гунзэчойнэй   

                              Долгота     Широта  \
Тип элемента Идентификатор                         
node         538426140      30.264162  59.997683   
             679667984      30.263726  60.013216   
             818774996      30.201097  60.009312   
             1066527082     30.232739  60.013229   
             1145235093     30.292656  59.989019   
...                               ...        ...   
way          103263767      30.131108  59.990024   
             113380025      29.965528  60.013289   
             163902553      30.200774  59.981524   
             566202597      30.147742  59.994195   
relation     1566440        30.256007  59.983441   

                                                                     geometry  
Тип элемента Идентификатор                                                     
node         538426140                              POINT (30.26416 59.99768)  
             679667984                              POINT (30.26373 60

In [78]:
# TODO сохранить слой в географической проекции в формате GeoJSON
#objects.to_crs(4326).to_file(KGIOP_FILE, driver='GeoJSON')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Тип  \
Тип элемента Идентификатор             
node         538426140      memorial   
             679667984      memorial   
             818774996      memorial   
             1066527082     memorial   
             1145235093     memorial   
...                              ...   
way          103263767         manor   
             113380025         ruins   
             163902553      monument   
             566202597           yes   
relation     1566440             yes   

                                                                      Надпись  \
Тип элемента Идентификатор                                                      
node         538426140      Военным морякам тихоокеанцам, погибшим при исп...   
             679667984      Площадь названа в честь авиаконструктора Игоря...   
             818774996                                                    NaN   
             1066527082                                                   NaN   
             1145235093                                                   NaN   
...                                                                       ...   
way          103263767                                                    NaN   
             113380025                                                    NaN   
             163902553                                                    NaN   
             566202597                                                    NaN   
relation     1566440                                                      NaN   

                           Материал  \
Тип элемента Идентификатор            
node         538426140        stone   
             679667984          NaN   
             818774996        stone   
             1066527082         NaN   
             1145235093         NaN   
...                             ...   
way          103263767          NaN   
             113380025          NaN   
             163902553          NaN   
             566202597          NaN   
relation     1566440            NaN   

                                                              Название  \
Тип элемента Идентификатор                                               
node         538426140                     Военным морякам-тихоокенцам   
             679667984                                И. И. Сикорскому   
             818774996              Закладной камень Аллеи журналистов   
             1066527082          Ликвидаторам Чернобыльской катастрофы   
             1145235093                  Камень памяти Ольги Берггольц   
...                                                                ...   
way          103263767      Охотничий домик графа В.А. Стенбок-Фермора   
             113380025                Пороховой погреб с лабораториями   
             163902553                                             NaN   
             566202597                                             NaN   
relation     1566440                            Храм Дацан Гунзэчойнэй   

                              Долгота     Широта  \
Тип элемента Идентификатор                         
node         538426140      30.264162  59.997683   
             679667984      30.263726  60.013216   
             818774996      30.201097  60.009312   
             1066527082     30.232739  60.013229   
             1145235093     30.292656  59.989019   
...                               ...        ...   
way          103263767      30.131108  59.990024   
             113380025      29.965528  60.013289   
             163902553      30.200774  59.981524   
             566202597      30.147742  59.994195   
relation     1566440        30.256007  59.983441   

                                                                     geometry  
Тип элемента Идентификатор                                                     
node         538426140                              POINT (30.26416 59.99768)  
             679667984                              POINT (30.26373 60